In [1]:
import gurobipy as gb
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from numpy import random
from scipy.spatial.distance import euclidean
from scipy.stats import truncnorm
import pickle
import os
from source import *

path = os.getcwd()

# Global variables

In [2]:
''' Non-linear charging function parameters '''
energy_bps = [0,212.5,237.5,250] # Energy level (miles) breakpoints in the piecewise linear function
charging_rates = [260.1,118.575,39.525] # Charging rates (miles/hour) for every piece in the charging function
full_rch_time = 1.344086022 # Time that it would take for a vehicle to recharge until full range (250 miles) with an empty battery


# Functions

In [3]:
def nl_charging_function(soc):

    p = [i for i in range(3) if energy_bps[i]<=soc and soc<=energy_bps[i+1]][0]
    return full_rch_time - sum((min(soc,energy_bps[pp+1])-energy_bps[pp])/charging_rates[pp] for pp in range(p+1))

def dump_parameters(sc,mu,sigma,K,r,S_k,K_s,tp,p,a,t):
    
    file = open(path+f"/Sensitivity/Data/K/K_mu{mu}_sigma{sigma}_sc{sc}", "wb")
    pickle.dump(K,file); file.close()

    file = open(path+f"/Sensitivity/Data/r/r_mu{mu}_sigma{sigma}_sc{sc}", "wb")
    pickle.dump(r,file); file.close()

    file = open(path + f'/Sensitivity/Data/S_k/Sk_mu{mu}_sigma{sigma}_sc{sc}', 'wb')
    pickle.dump(S_k, file); file.close()

    file = open(path + f'/Sensitivity/Data/K_s/Ks_mu{mu}_sigma{sigma}_sc{sc}', 'wb')
    pickle.dump(K_s, file); file.close()

    file = open(path + f"/Sensitivity/Data/tp/tp_mu{mu}_sigma{sigma}_sc{sc}", "wb")
    pickle.dump(tp,file); file.close()

    file = open(path + f'/Sensitivity/Data/p/p_mu{mu}_sigma{sigma}_{sc}', 'wb')
    pickle.dump(p, file); file.close()

    file = open(path + f'/Sensitivity/Data/a t/at_mu{mu}_sigma{sigma}_{sc}', 'wb')
    pickle.dump((a,t), file); file.close()


def generate_stochastic_parameters(mu, sigma, distances, S):

    ''' Import vehicles dataframe'''
    vehicles = pd.read_csv(path+"/Data/Definite Vehicles.csv",index_col=[0])

    ''' Demand parameters for truncated normal distribution '''
    loc, scale, min_v, max_v = mu, sigma, 20, 250
    aa, b = (min_v - loc) / scale, (max_v - loc) / scale

    ''' Import other parameters '''
    file = open(path+"/Data/Global/dem_distribution","rb")
    dem_distribution = pickle.load(file); file.close()
    dem_time_steps = list(range(len(dem_distribution)))

    ''' Scenario generation '''
    flag = True
    ii = -1
    while flag:
        ii += 1
        flag = False
        ranges, realized = dict(), dict()

        for i in vehicles.index:
            feas = False
            rr = truncnorm.rvs(a = aa, b = b, loc = loc, scale = scale, size = 1)
            ranges[i] = rr[0]

            prob = np.exp(-0.012**2*(rr-20)**2)
            realization = random.choice([True,False], p = [prob[0], 1-prob[0]])
            realized[i] = realization

            if realization == True:
                for st in S:
                    if distances[st,i] <= rr:
                        feas = True; break

                if not feas:
                    flag = True
                    print(f'\tAttempt n. {ii} failed at vehicle {i}')
                    break
    
    K = [k for k in vehicles.index if realized[k]]
    r = {k:ranges[k] for k in K}
    tp = {k:random.choice(dem_time_steps,p=dem_distribution) for k in K}

    return K, r, tp

def get_optimal_solution():
    file = open(path+"/Results/Optimal/S","rb")
    S = pickle.load(file); file.close()

    file = open(path+"/Results/Optimal/n","rb")
    n = pickle.load(file); file.close()

    file = open(path+"/Results/Optimal/distances","rb")
    distances = pickle.load(file); file.close()

    return S, n, distances

def run_scenario(mu,sigma,sc):

    ''' Import station configuration, number of chargers and distance matrix '''
    S, n, distances = get_optimal_solution()

    ''' Import global parameters '''
    file = open(path+"/Data/Global/driving_speed","rb")
    driving_speed = pickle.load(file); file.close()
    
    ''' Generate stochastic parameters of scenario '''
    K, r, tp = generate_stochastic_parameters(mu,sigma,distances,S)
    
    ''' Compute other parameters '''
    S_k = {k:[s for s in S if distances[s,k]<=r[k]] for k in K}
    K_s = {s:[k for k in K if s in S_k[k]] for s in S}

    for k in K:
        dist = {st:distances[st,k] for st in S_k[k]}
        S_k[k] = [st for st in sorted(dist, key=dist.get, reverse=False)[:min(10,len(S_k[k]))]]
    K_s = {s:[k for k in K_s[s] if s in S_k[k]] for s in S}
    
    p = {(k,s):250-(r[k]-distances[s,k]) for s in S for k in K_s[s]}
    t = {(k,s):nl_charging_function(r[k]-distances[s,k])  for s in S for k in K_s[s]}
    a = {(k,s):distances[s,k]/driving_speed+tp[k]/4 for s in S for k in K_s[s]}
    a.update({("s",s):0 for s in S}); a.update({("e",s):30 for s in S})
    t.update({("s",s):0 for s in S}); t.update({("e",s):0 for s in S})

    ''' Saves scenario parameters '''
    dump_parameters(sc,mu,sigma,K,r,S_k,K_s,tp,p,a,t)

    results = test_scenario_sensitivity(S,K,K_s,n,a,t)

    print("\n--------------------RESULTS--------------------")
    print(f"A total of {len(K)} vehicles need to be assigned")
    print(f"\tTotal assigned vehicles: {len(results['total_total'])}")
    print(f"\tTotal unassigned vehicles: {len(results['infeasible'])}")
    print(f"Achieved Service Level: {round(len(results['infeasible'])/len(K),2)}")

    return results


In [4]:
results = run_scenario(100,50,0)

	Attempt n. 0 failed at vehicle 8614
	Attempt n. 1 failed at vehicle 5369
	Attempt n. 2 failed at vehicle 6414
	Attempt n. 3 failed at vehicle 5364
Set parameter Username
Academic license - for non-commercial use only - expires 2024-01-08
		Iteration 0:		MP obj: 4556.0	time: 0.05s


KeyboardInterrupt: 